In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/bofenghuang/vigogne.git
%cd vigogne

In [ ]:
# Install requirements
!pip install -r requirements.txt

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer

/home/bhuang/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/bhuang/anaconda3/envs/nlp/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/bhuang/anaconda3/envs/nlp/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/bhuang/anaconda3/envs/nlp did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [3]:
base_model_name_or_path = "decapoda-research/llama-7b-hf"
lora_model_name_or_path = "bofenghuang/vigogne-lora-7b"

# base_model_name_or_path = "decapoda-research/llama-30b-hf"
# lora_model_name_or_path = "bofenghuang/vigogne-lora-30b"

# base_model_name_or_path = "bigscience/bloom-7b1"
# lora_model_name_or_path = "bofenghuang/vigogne-lora-bloom-7b1"

load_8bit = False

tokenizer_class = LlamaTokenizer if "llama" in base_model_name_or_path else AutoTokenizer
tokenizer = tokenizer_class.from_pretrained(base_model_name_or_path)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name_or_path,
        torch_dtype=torch.float16,
    )
elif device == "mps":
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name_or_path,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name_or_path,
        device_map={"": device},
    )

if not load_8bit:
    model.half()  # seems to fix bugs for some users.

model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Loading checkpoint shards: 100%|██████████| 33/33 [00:39<00:00,  1.20s/it]


In [4]:
PROMPT_DICT = {
    "prompt_input": (
        "Ci-dessous se trouve une instruction qui décrit une tâche, associée à une entrée qui fournit un contexte supplémentaire. Écrivez une réponse qui complète correctement la demande.\n\n"
        "### Instruction:\n{instruction}\n\n### Entrée:\n{input}\n\n### Réponse:\n"
    ),
    "prompt_no_input": (
        "Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète correctement la demande.\n\n"
        "### Instruction:\n{instruction}\n\n### Réponse:\n"
    ),
}


def generate_prompt(instruction, input=None):
    return (
        PROMPT_DICT["prompt_input"].format_map({"instruction": instruction, "input": input})
        if input is not None
        else PROMPT_DICT["prompt_no_input"].format_map({"instruction": instruction})
    )


def instruct(
    instruction,
    input=None,
    temperature=0.1,
    no_repeat_ngram_size=3,
    max_new_tokens=512,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    tokenized_inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = tokenized_inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        no_repeat_ngram_size=no_repeat_ngram_size,
        **kwargs,
    )
    with torch.inference_mode():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            # output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    return output.split("### Réponse:")[1].strip()

In [5]:
# instruct
instruct("Expliquer le théorème central limite.")

'Le théorême central limites affirme que la loi de la moyenne des valeurs de variables aléatoires indépendantes est la loi normale. Cela signifie que la distribution des valeur de la somme de plusieurs variables al équivalentes est la même que la normale, et que la variance de la distribution de la valeur moyenne de plusieurs de ces variables est la variance du résultat de la loi des grands nombres.'

In [6]:
# instruct + input
instruct(
    "Traduisez le texte suivant en français.",
    input="Caterpillars extract nutrients which are then converted into butterflies. People have extracted billions of nuggets of understanding and GPT-4 is humanity's butterfly.",
)

"Les papillons extraient des nutriments qui sont ensuite convertis en papillones. Les gens ont extrait des milliards de nugget de compréhension et GPT4 est la butterlfly de l'humanité."